In [1]:
from music21 import converter, environment, tempo, metadata
import os
import glob
from PIL import Image

us = environment.UserSettings()
us['musicxmlPath'] = '/usr/bin/musescore3'

print(environment.UserSettings())

{'directoryScratch': None, 'lilypondPath': '/usr/bin/lilypond', 'lilypondVersion': None, 'lilypondFormat': 'pdf', 'lilypondBackend': 'ps', 'musicxmlPath': '/usr/bin/musescore3', 'midiPath': None, 'graphicsPath': '/usr/bin/xdg-open', 'vectorPath': None, 'pdfPath': '/usr/bin/xdg-open', 'braillePath': None, 'musescoreDirectPNGPath': '/usr/bin/mscore3', 'showFormat': 'musicxml', 'writeFormat': 'musicxml', 'ipythonShowFormat': 'ipython.musicxml.png', 'autoDownload': 'ask', 'debug': '0', 'warnings': '1', 'localCorpusSettings': LocalCorpusSettings([]), 'localCorporaSettings': {}, 'manualCoreCorpusPath': None}


In [2]:
def add_margins_and_white_bg(image_path, top=20, right=60, bottom=20, left=60):
    img = Image.open(image_path).convert("RGBA")
    new_width = img.width + left + right
    new_height = img.height + top + bottom
    background = Image.new("RGBA", (new_width, new_height), (255, 255, 255, 255))
    background.paste(img, (left, top), mask=img)
    background.convert("RGB").save(image_path, "PNG")

def fix_all_images(png_prefix):
    png_files = sorted(glob.glob(f"{png_prefix}-*.png"))
    if os.path.exists(f"{png_prefix}.png"):
        png_files.insert(0, f"{png_prefix}.png")
    
    for path in png_files:
        add_margins_and_white_bg(path)



In [3]:
midi_path = '../data/Boccherini-Minuet-Flute-and-Piano.mid'
score = converter.parse(midi_path)

tempi = list(score.recurse().getElementsByClass(tempo.MetronomeMark))
if len(tempi) > 1:
    for tm in tempi[1:]:
        parent = tm.getContextByClass('Measure')
        if parent:
            parent.remove(tm)
        else:
            tm.activeSite.remove(tm)

filename = os.path.splitext(os.path.basename(midi_path))[0]
if score.metadata is None:
    score.metadata = metadata.Metadata()
score.metadata.title = filename


output_path = f"../output/{filename}"
score.write('musicxml.png', fp=f"{output_path}.png")

fix_all_images(output_path)

